In [23]:
import pandas as pd
import yaml, os, openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_csv_agent
from langchain import PromptTemplate, FewShotPromptTemplate

In [24]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

# os.environ['OPENAI_API_KEY'] = credentials['AZURE_OPENAI_KEY']
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']

# openai.api_key = credentials['AZURE_OPENAI_KEY']
# openai.api_base = credentials['AZURE_OPENAI_BASE']

In [25]:
llm = ChatOpenAI(
                openai_api_key=os.environ["OPENAI_API_KEY"],
                engine = os.environ["ENGINE"],
                model='gpt-3.5-turbo',
                temperature=0, 
                max_tokens=256
                )

# llm = ChatOpenAI(
#                 openai_api_key=os.environ["OPENAI_API_KEY"],
#                 model='gpt-3.5-turbo',
#                 temperature=0, 
#                 max_tokens=256
#                 )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [26]:
df = pd.read_csv('data/12-black-friday.csv')
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


## CSV Agent

NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.

In [27]:
# !pip install tabulate

In [28]:
agent = create_csv_agent(
                        llm, 
                        'data/12-black-friday.csv', 
                        verbose=True
                        )


In [29]:
agent

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], partial_variables={'df_head': '|    |   User_ID | Product_ID   | Gender   | Age   |   Occupation | City_Category   | Stay_In_Current_City_Years   |   Marital_Status |   Product_Category_1 |   Product_Category_2 |   Product_Category_3 |   Purchase |\n|---:|----------:|:-------------|:---------|:------|-------------:|:----------------|:-----------------------------|-----------------:|---------------------:|---------------------:|---------------------:|-----------:|\n|  0 |   1000001 | P00069042    | F        | 0-17  |           10 | A               | 2                            |                0 |                    3 |                  nan |                  nan |       8370 |\n|  1 |   1000001 | P00248942    | F        | 0-17  |           10 | A               | 2                            |                0 |                    1 |                  

In [30]:
print(agent.agent.llm_chain.prompt.template)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}


In [31]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `shape` attribute of the dataframe, which returns a tuple with the number of rows and columns.

Action: python_repl_ast
Action Input: df.shape[0]
Observation: 550068
Thought:There are 550068 rows in the dataframe.
Final Answer: 550068

> Finished chain.


'550068'

In [32]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows where the Gender column is "F".
Action: Use the `value_counts()` method on the Gender column to count the occurrences of each value.
Action Input: `df['Gender'].value_counts()`
Observation: Use the `value_counts()` method on the Gender column to count the occurrences of each value. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.
Action: python_repl_ast
Action Input: `df['Gender'].value_counts()`
Observation: Gender
M    414259
F    135809
Name: count, dtype: int64
Thought:There are 135,809 people who are female.
Final Answer: 135,809

> Finished chain.


'135,809'

In [34]:
agent.run("how many people have stayed 3> years in the city?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows where the value in the "Stay_In_Current_City_Years" column is greater than 3.

Action: Use the `value_counts()` method to count the occurrences of each unique value in the "Stay_In_Current_City_Years" column.

Action Input: `df["Stay_In_Current_City_Years"].value_counts()`

Observation: Use the `value_counts()` method to count the occurrences of each unique value in the "Stay_In_Current_City_Years" column. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: `python_repl_ast`

Action Input: `df["Stay_In_Current_City_Years"].value_counts()`

Observation: `python_repl_ast` is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: `python_repl_ast`

Action Input: `df["Stay_In_Current_City_Years"].value_counts()`

Observation: `python_repl_ast` is not a v

'Agent stopped due to iteration limit or time limit.'